# Datamart
This notebook aims to restrict data perimeter due to the large amount of the input data

In [51]:
import pandas as pd
import numpy as np
import os
import pyspark
import pyspark.sql.functions as F

## perimeter
Movies produced between 2010 & 2019

In [52]:
title_basics = pd.read_parquet("../data/parsed_data/imdb.db/title_basics")

In [53]:
title_perimeter = title_basics[(title_basics.titleType == "movie")][["tconst"]]

In [54]:
title_perimeter.shape

(528662, 1)

In [55]:
title_perimeter.to_parquet("../data/edited_data/datamart.db/title_perimeter")

## Datamart creation
### link.name.basics.title

In [56]:
perimeter = pd.read_parquet("../data/edited_data/datamart.db/title_perimeter")
link_name_basics_titles = pd.read_parquet("../data/parsed_data/imdb.db/link_name_basics_titles")

In [57]:
link_name_basics_titles = (
    link_name_basics_titles
    .merge(perimeter, on="tconst", how="inner")
    .drop(columns=["tconst"], axis=1)
    .dropna()
    .drop_duplicates()
    .merge(link_name_basics_titles, on="nconst", how="inner")
)

In [58]:
print(link_name_basics_titles.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9134387 entries, 0 to 9134386
Data columns (total 2 columns):
nconst    object
tconst    object
dtypes: object(2)
memory usage: 209.1+ MB
None


In [59]:
link_name_basics_titles.to_parquet("../data/edited_data/datamart.db/link_name_basics_titles")

### titles: all the movies in which an actor who played in the movie perimeter played in

In [60]:
link_name_basics_titles = pd.read_parquet("../data/edited_data/datamart.db/link_name_basics_titles")

In [61]:
titles = link_name_basics_titles[["tconst"]].drop_duplicates()

In [62]:
titles.shape

(1047435, 1)

In [63]:
titles.to_parquet("../data/edited_data/datamart.db/titles")

#### names

In [64]:
link_name_basics_titles = pd.read_parquet("../data/edited_data/datamart.db/link_name_basics_titles")

In [65]:
names = link_name_basics_titles[["nconst"]].drop_duplicates()

In [66]:
names.shape

(4078391, 1)

In [67]:
names.to_parquet("../data/edited_data/datamart.db/names")

#### title.basics

In [68]:
title_basics = pd.read_parquet("../data/parsed_data/imdb.db/title_basics")
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")

In [69]:
title_basics = title_basics.merge(titles, on="tconst", how="inner")

In [70]:
title_basics.to_parquet("../data/edited_data/datamart.db/title_basics")

#### title.akas

In [71]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_akas = pd.read_parquet("../data/parsed_data/imdb.db/title_akas")

In [72]:
title_akas = title_akas.merge(titles.rename(columns={"tconst": "titleId"}), on="titleId", how="inner")

In [73]:
title_akas.shape

(2396355, 8)

In [74]:
title_akas.to_parquet("../data/edited_data/datamart.db/title_akas")

#### title.ratings

In [75]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_ratings = pd.read_parquet("../data/parsed_data/imdb.db/title_ratings")

In [76]:
title_ratings = title_ratings.merge(titles, on="tconst", how="inner")

In [77]:
titles.shape, title_ratings.shape

((1047435, 1), (421056, 3))

In [78]:
title_ratings.to_parquet("../data/edited_data/datamart.db/title_ratings")

#### name.basics

In [79]:
names = pd.read_parquet("../data/edited_data/datamart.db/names")
name_basics = pd.read_parquet("../data/parsed_data/imdb.db/name_basics")

In [80]:
name_basics = name_basics.merge(names, on="nconst", how="inner")

In [81]:
name_basics.shape

(4078391, 6)

In [82]:
name_basics.to_parquet("../data/edited_data/datamart.db/name_basics")

#### title.crew 

In [95]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_crew = pd.read_parquet("../data/parsed_data/imdb.db/title_crew")

In [99]:
title_crew = title_crew.merge(titles, on="tconst", how="inner")

,tconst,directors,writers
0,tt0000001,nm0005690,None
1,tt0000009,nm0085156,nm0085156
2,tt0000024,nm0010291,None
3,tt0000026,nm0525910,None
4,tt0000029,nm0525910,None
...,...,...,...
1046954,tt9916706,nm7764440,nm7933903
1046955,tt9916720,nm10538600,None
1046956,tt9916730,nm10538612,nm10538612
1046957,tt9916754,"nm9272490,nm8349149","nm8349149,nm9272490"


In [85]:
title_crew.shape

(1046959, 3)

In [86]:
title_crew.to_parquet("../data/edited_data/datamart.db/title_crew")

#### link.title.directors

In [87]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
link_title_directors = pd.read_parquet("../data/parsed_data/imdb.db/link_title_directors")

In [88]:
link_title_directors = link_title_directors.merge(titles, on="tconst", how="inner")

In [89]:
link_title_directors.shape

(1267915, 2)

In [90]:
link_title_directors.to_parquet("../data/edited_data/datamart.db/link_title_directors")

#### title.principals

In [91]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_principals = pd.read_parquet("../data/parsed_data/imdb.db/title_principals")

In [92]:
title_principals = title_principals.merge(titles, on="tconst", how="inner")

In [93]:
title_principals.shape

(7727196, 6)

In [94]:
title_principals.to_parquet("../data/edited_data/datamart.db/title_principals")